### - - 이전 내용 코드 시작 - - 

In [1]:
#코드 4-2 의존성 패키지 설치
!pip install ratsnlp

     |████████████████████████████████| 42 kB 112 kB/s 
     |████████████████████████████████| 2.8 MB 8.7 MB/s 
     |████████████████████████████████| 806 kB 37.9 MB/s 
     |████████████████████████████████| 57 kB 5.3 MB/s 
     |████████████████████████████████| 636 kB 33.9 MB/s 
     |████████████████████████████████| 397 kB 30.3 MB/s 
     |████████████████████████████████| 134 kB 16.1 MB/s 
     |████████████████████████████████| 829 kB 32.9 MB/s 
     |████████████████████████████████| 895 kB 33.2 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 3.3 MB 34.1 MB/s 
     |████████████████████████████████| 1.1 MB 42.6 MB/s 
     |████████████████████████████████| 96 kB 5.8 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 271 kB 49.2 MB/s 
     |████████████████████████████████| 144 kB 43.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha

In [2]:
#코드 4-3 구글 드라이브와 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
#코드 4-4 모델 환경 설정
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/My drive/nlpbook/checkpoint-doccls",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [4]:
#코드 4-5 랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [5]:
#코드 4-6 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My drive/nlpbook/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)


In [6]:
#코드 4-7 말뭉치 내려받기
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 86.1MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 36.9MB/s]                            


In [7]:
#코드 4-8 토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [8]:
#코드 4-9 학습 데이터셋 구축
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 41.349 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: [CLS] 아 더 ##빙 . . 진짜 짜증나네 ##요 목소리 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [9]:
train_dataset[0]

ClassificationFeatures(input_ids=[2, 2170, 832, 5045, 17, 17, 7992, 29734, 4040, 10720, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [10]:
train_dataset[100]

ClassificationFeatures(input_ids=[2, 2005, 4024, 4017, 1293, 4599, 4775, 4042, 2478, 4075, 4196, 15, 1463, 4207, 4196, 8080, 4024, 10314, 11219, 4180, 12610, 10579, 832, 4140, 11414, 9827, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
#코드 4-10 학습 데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers
)

In [12]:
#코드 4-11 평가용 데이터 로더 구축
from torch.utils.data import SequentialSampler
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 14.115 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 굳 ㅋ
INFO:ratsnlp:tokens: [CLS] 굳 ㅋ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

### - - 이전 내용 코드 끝 - - 

<**6단계**> **모델 불러오기**  
이제 모델을 초기화하는 다음 코드를 실행합니다. 코드 4-4에서 **pretrained_model_name**을 **beomi/kcbert-base**로 지정했으므로 프리트레인을 마친 BERT로 **kcbert-base**를 사용합니다. 모델을 초기화하는 코드에서 **BertForSequenceClassification**은 프리트레인을 마친 BERT 모델 위에 <4-1>절에서 설명한 문서 분류용 태스크 모듈이 덧붙여진 형태의 모델 클래스입니다. 이 클래스는 허깅페이스에서 제공하는 **transformers** 라이브러리에 포함돼 있습니다.

In [13]:
#코드 4-12 모델 초기화
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

허깅페이스 모델 허브에 등록된 모델이라면 별다른 코드 수정 없이 **kcbert-base** 이외에 다른 언어 모델을 사용할 수 있습니다. 예를 들어 **bert-base-uncased** 모델은 구글이 공개한 다국어 BERT 모델인데요, 코드 4-4에서 **pretrained_model_name**에 이 모델명을 입력하면 해당 모델을 쓸 수 있습니다. 허깅페이스에 등록된 모델 목록은 huggingface.co/models에서 확인할 수 있습니다.  
아울러 코드 4-8, 4-12에는 똑같은 모델 이름을 입력해야 합니다.

<**7단계**> **모델 학습시키기**  

파이토치 라이트닝(pytorch lighting)이 제공하는 **LightingModule** 클래스를 상속받아 태스크(task)를 정의합니다. 태스크에는 다음 그림처럼 모델과 옵티마이저, 학습 과정 등이 정의돼 있습니다. 

<center><그림 4-6 TASK의 역할></center>

<p align="center"><img src="https://i.imgur.com/ViOHWFw.jpg">  

<center>출처 : ratsgo's NLPBOOK</center>


다음 코드를 실행하면 문서 분류용 태스크를 정의할 수 있습니다. 코드 4-4에서 만든 학습 설정(**args**)과 코드 4-12에서 준비한 모델(**model**)을 **ClassificationTask**에 주입합니다. **ClassificationTask**에는 **옵티마이저(optimizer)**, **러닝 레이트 스케줄러(learning rate scheduler)**가 정의돼 있는데요, 옵티마이저로는 **아담(Adam)**, 러닝 레이트 스케줄러로는 **ExponentialLR**을 사용합니다.

In [14]:
#코드 4-13 TASK 정의
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

모델 학습 과정은 눈을 가린 상태에서 산등성이를 한 걸음씩 내려가는 과정에 비유할 수 있습니다. 러닝 레이트는 한 번 내려갈 때 얼마나 이동할지 보폭에 해당합니다. 학습이 진행되는 동안 점차 러닝 레이트를 줄여 세밀하게 탐색하면 좀 더 좋은 모델을 만들 수 있습니다. 이 역할을 하는 게 바로 러닝 레이트 스케줄러입니다. **ExponentialLR**은 현재 에포크의 러닝 레이트를 '이전 에포크의 러닝 레이트 $\times$ **gamma**'로 스케줄링합니다.  
이 책의 실습에서는 **gamma**를 0.9로 두고 있습니다.

**ClassificationTask**의 자세한 구현 내용에 대해서는 다음 링크를 참고하세요. 학습 및 평가 과정에서 미니 배치가 입력됐을 때 손실(loss)을 계산하는 과정, 옵티마이저와 러닝 레이트 스케줄러를 정의하는 내용 등을 소개합니다.

- ratsgo.github.io/nlpbook/docs/doc_cls/detail

코드 4-14를 실행하면 트레이너를 정의할 수 있습니다. 이 트레이너는 파이토치 라이트닝 라이브러리의 도움을 받아 GPU/TPU 설정, 로그 및 체크포인트 등 귀찮은 설정들을 알아서 해줍니다.

In [15]:
#코드 4-14 트레이너 정의
trainer = nlpbook.get_trainer(args)

OSError: ignored

In [17]:
!pwd

/content


In [20]:
!cd '/gdrive/My drive'

/bin/bash: line 0: cd: /gdrive/My drive: No such file or directory


다음 코드처럼 트레이너의 **fit()** 함수를 호출하면 학습을 시작합니다. 학습 시간은 자신의 코랩 환경에 따라 다를 수 있으나 꽤 오래 걸릴 수 있습니다. 1에포크 학습에 필요한 예상 시간은 그림처럼 확인할 수 있습니다. 학습이 진행되는 도중 브라우저를 끄면 모델 학습을 비롯한 코랩 실행이 중단되니 주의하세요.

In [16]:
#코드 4-15 학습 개시
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

NameError: ignored

참고로 코랩 정책이 변경되거나 계정 종류에 따라서 학습을 다 마치지 않았는데 실행이 중단될 수도 있습니다. 하지만 이 경우라도 코드 4-4의 **args.downstream_mode_dir**에 해당하는 디렉터리에 모델 체크포인트(확장자가 ckpt인 파일)가 하나라도 있으면 다음 절 실습을 진행할 수 있습니다. 이 실습 코드에서는 각 에포크 종료 시 평가용 데이터로 모델의 성능을 측정한 뒤 이전보다 성능이 개선되었을 경우 모델 체크포인트를 저장합니다. 코드 4-4의 **args.epochs**를 1이나 2로 줄여서 학습 시간을 단축하는 것도 방법입니다.

지금까지 우리는 문서 분류 모델 학습을 실습해 보았습니다. 큰 줄기는 이렇습니다. 각종 설정을 마친 뒤 말뭉치를 내려받아 전처리합니다. 프리트레인을 마친 모델을 이 데이터에 맞게 파인튜닝합니다. 이 장 실습은 이후 다른 장 실습의 기본이 되니 잘 숙지해 두면 좋습니다. 특히 데이터셋, 데이터 로더 등 개념에 익숙해진다면 더할 나위 없을 듯합니다.

<center><그림 4-7 코랩 환경에서의 학습></center>

<p align="center"><img src="https://i.imgur.com/dcKpN4U.png">  

<center>출처 : ratsgo's NLPBOOK</center>